Scraping College Stats

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

In [15]:
file = 'C:\\Users\\Chung-Tseng Wang\\Desktop\\Github Repo\\Predicting-NBA-Rookie-Stats\\data\\processed\\processed_nba.csv'
nba = pd.read_csv(file, index_col = 'Player')
nba.head()

,year,Player.1,id,Age,G,FG%,3P%,FT%,PS/G,TRB/G,AST/G,STL/G,BLK/G,TOV/G
Player,,,,,,,,,,,,,,
Jaylen Adams,2019,Jaylen Adams,adamsja01,22,34,0.345,0.338,0.778,3.2,1.8,1.9,0.411765,0.147059,0.823529
Deng Adel,2019,Deng Adel,adelde01,21,19,0.306,0.261,1.000,1.7,1.0,0.3,0.052632,0.210526,0.315789
DeVaughn Akoon-Purcell,2019,DeVaughn Akoon-Purcell,akoonde01,25,7,0.300,0.000,0.500,1.0,0.6,0.9,0.285714,0.000000,0.285714
Rawle Alkins,2019,Rawle Alkins,alkinra01,21,10,0.333,0.250,0.667,3.7,2.6,1.3,0.100000,0.000000,0.800000
Grayson Allen,2019,Grayson Allen,allengr01,23,38,0.376,0.323,0.750,5.6,0.6,0.7,0.157895,0.157895,0.868421


In [14]:
players_list = nba.index.values
len(players_list)

892

In [39]:
data = []
for player in players_list:
    url_name = player
    player = player.lower()
    player = player.replace('.','')
    player = player.replace("'",'')
    player = player.replace(' ','-')
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ player +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    table = soup.find('table',{'id':'players_per_game'})
    if table is not None:
        data = data + [url_name]
        stats = table.findAll('td')
        elements = []
        for n in stats:
            elements.append(n.get_text())
        data = data + elements[-28:]

In [34]:
transformed = np.reshape(np.array(data),(-1,29))

In [58]:
#To get column headers
headers = ['Player','School','Conf','G','GS','MP','FG','FGA','FGP','2P','2PA','2PP','3P','3PA','3P%','FT','FTA','FTP','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','nan','SOS']
college = pd.DataFrame(transformed, columns = headers)
college = college.drop(['Conf','GS','MP','FG','FGA','2P','2PA','2PP','3P','3PA','FT','FTA','ORB','DRB','nan','SOS','PF'], axis = 1)
college.head()

,Player,School,G,FGP,3P%,FTP,TRB,AST,STL,BLK,TOV,PTS
0,Jaylen Adams,St. Bonaventure,110,.427,.394,.838,3.4,5.4,1.5,0.2,2.9,17.4
1,Deng Adel,Louisville,89,.438,.348,.776,4.1,2.0,0.5,0.3,1.7,11.2
2,DeVaughn Akoon-Purcell,Illinois State,60,.428,.301,.739,5.3,1.6,1.0,0.3,2.3,13.5
3,Rawle Alkins,Arizona,60,.450,.365,.729,4.9,2.2,1.0,0.6,2.1,11.8
4,Grayson Allen,Duke,142,.430,.380,.834,3.2,3.0,1.0,0.1,1.7,14.1


In [59]:
college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 12 columns):
Player    738 non-null object
School    738 non-null object
G         738 non-null object
FGP       738 non-null object
3P%       738 non-null object
FTP       738 non-null object
TRB       738 non-null object
AST       738 non-null object
STL       738 non-null object
BLK       738 non-null object
TOV       738 non-null object
PTS       738 non-null object
dtypes: object(12)
memory usage: 69.3+ KB


In [60]:
college.describe()

,Player,School,G,FGP,3P%,FTP,TRB,AST,STL,BLK,TOV,PTS
count,738,738,738,738,738,738,738,738,738,738,738,738
unique,735,191,139,247,221,313,109,70,30,39,45,187
top,Chris Johnson,Kentucky,35,.450,,.733,4.8,1.5,0.9,0.1,1.5,14.1
freq,2,37,18,14,56,9,22,35,63,94,51,13


In [65]:
college.to_csv('raw_college.csv', index = False)